# 4.2 VectorDBBench 基准测试实战

本节将带你了解并实践 VectorDBBench 这一主流向量数据库基准测试工具的部署与使用。

## 4.2.1 VectorDBBench 简介

VectorDBBench（VDBBench）是一个面向主流向量数据库和云服务的开源基准测试工具，支持多种数据库的性能与性价比对比，提供可视化界面和丰富的测试场景，方便用户复现结果或测试新系统。

- 支持多种数据库（如 Milvus、Zilliz Cloud、Qdrant、Weaviate、PgVector、Redis、Chroma 等）
- 提供插入、搜索、过滤搜索、流式搜索等多种测试场景
- 内置多种公开数据集（如 SIFT、GIST、Cohere、OpenAI C4 等）
- 支持可视化界面，便于配置测试、查看和对比测试结果

主要特性：
1. 简单易用的 Web UI，支持测试配置和结果可视化分析
2. 标准化的测试流程和指标采集，支持多场景扩展（如过滤、流式）
3. 支持多种主流和新兴向量数据库，便于横向对比

更多介绍详见 [官方文档](https://github.com/zilliztech/VectorDBBench)

## 4.2.2 VectorDBBench 部署（多种数据库的 client）

### 环境要求
- Python >= 3.11

### 安装
仅安装 Milvus/Zilliz Cloud 客户端：
```shell
pip install vectordb-bench
```

安装所有支持的数据库客户端（如需对比多种数据库）：
```shell
pip install vectordb-bench[all]
```

安装指定数据库客户端（如 Qdrant）：
```shell
pip install vectordb-bench[qdrant]
```

支持的数据库客户端及安装命令如下：

| 数据库客户端 | 安装命令 |
|-------------|----------|
| pymilvus, zilliz_cloud | `pip install vectordb-bench` |
| all         | `pip install vectordb-bench[all]` |
| qdrant      | `pip install vectordb-bench[qdrant]` |
| pinecone    | `pip install vectordb-bench[pinecone]` |
| weaviate    | `pip install vectordb-bench[weaviate]` |
| elastic, aliyun_elasticsearch | `pip install vectordb-bench[elastic]` |
| pgvector, pgvectorscale, pgdiskann, alloydb | `pip install vectordb-bench[pgvector]` |
| redis       | `pip install vectordb-bench[redis]` |
| chromadb    | `pip install vectordb-bench[chromadb]` |
| awsopensearch | `pip install vectordb-bench[opensearch]` |
| oceanbase   | `pip install vectordb-bench[oceanbase]` |
| ...         | ... |

更多数据库支持和安装方式详见官方 README。

## 4.2.3 VectorDBBench 网页启动及功能介绍

### 启动 Web 界面

安装完成后，直接运行：
```shell
init_bench
```
或
```shell
python -m vectordb_bench
```

默认会启动本地 Web 服务（如 http://localhost:8501），打开浏览器即可访问。

### 主要功能模块
- **Run Test**：选择数据库、填写连接信息、选择测试用例，发起基准测试。支持多数据库、多用例、多数据集组合测试。
- **Result**：查看所有测试结果，支持多轮对比和筛选，支持 QPS、延迟、性价比等多维度展示。
- **Custom Dataset**：自定义数据集和测试用例，支持详细参数配置（如维度、数据量、标签分布等）。
- **Quries Per Dollar**：展示每美元可处理的查询量，便于性价比分析。
- **Tables**：以表格形式对比不同数据集下的各项指标。
- **Concurrent Performance**：展示不同并发下 QPS 与延迟的变化趋势。
- **Label Filter Performance**：展示不同标签过滤比例下的性能表现。
- **Int Filter Performance**：展示不同整数过滤比例下的性能表现。
- **Streaming Performance**：展示在持续插入压力下的检索性能。

#### 典型界面示例

![VDBBench 主界面](./images/VectorDBBench.png)

![VDBBench 结果页面](https://github.com/zilliztech/VectorDBBench/assets/105927039/8a981327-c1c6-4796-8a85-c86154cb5472)

## 4.2.4 使用默认数据集进行基准测试

### 测试流程说明
VDBBench 标准测试流程分为三个主要阶段：

1. **Load（插入+优化）**：单进程串行插入全部数据，记录插入耗时（insert_duration）；部分数据库还会进行索引优化，记录优化耗时（optimize_duration）。总耗时（load_duration）反映数据库从零到可查询的整体加载能力。
2. **Serial Search Test（串行检索）**：单进程串行检索，记录每次查询的召回率（recall）和延迟（latency_p99）。p99 延迟关注最慢的 1% 请求，适合高要求场景。
3. **Concurrent Search Test（并发检索）**：多进程并发检索，逐步提升并发度（如 1~80），每组运行 30 秒，记录不同并发下的 QPS 和延迟，最终取最大 QPS 作为 max-qps。

此外还支持：
- **Filter Search Test**：在检索时增加标签或整数过滤条件，考察不同过滤比例下的性能变化。
- **Streaming Search Test**：在持续插入压力下分阶段进行检索，考察数据库在流式写入场景下的检索能力。

### 实践步骤
1. 启动 Web 界面后，进入 **Run Test** 页面
2. 选择要测试的数据库系统（如 Milvus、Qdrant、PgVector 等），填写连接信息
3. 选择测试用例（如 Capacity、Performance、Filtering、Streaming 等），可多选
4. 选择默认数据集（如 SIFT、GIST、Cohere、OpenAI C4 等）
5. 填写 Task Label，点击提交，等待测试完成

### 注意事项
- 默认数据集已内置，无需手动上传
- 可根据实际需求选择不同的测试用例和数据规模
- 建议测试客户端与数据库服务部署在同一局域网，减少网络延迟影响

测试完成后，可在 **Result** 页面查看详细结果和对比分析

## 4.2.5 结果解析

### 结果页面说明
- 每次测试结果会自动保存，可多轮对比
- 支持按 QPS、延迟、性价比等多维度排序和筛选
- 可查看每个用例的详细指标（如插入速度、查询延迟、召回率等）
- 支持 Quries Per Dollar、并发性能、过滤性能、流式性能等多维度可视化分析

### 评分规则简述
- QPS/QP$：分数按相对最高值归一化
- Latency：分数按最低延迟归一化（加 10ms 平滑）
- 失败/超时：分数按最差结果的两倍惩罚
- 综合分：各用例分数的几何平均值

#### 主要指标说明
- **insert_duration / optimize_duration / load_duration**：反映数据加载能力
- **recall**：召回率，衡量检索准确性
- **latency_p99**：99 分位延迟，关注极端慢请求
- **max-qps**：最大吞吐能力
- **filter/concurrent/streaming**：多维度场景下的性能表现

详细规则可参考 [Leaderboard 说明](https://zilliz.com/benchmark)

### 示例
![Leaderboard 示例](https://zilliz.com/benchmark)

通过这些结果，可以直观对比不同数据库在不同场景下的性能表现，辅助选型和调优。